# Data AugmentationによるCNNの精度向上

##本チュートリアルではchainerを利用してニューラルネットワークの実装を確認，学習および評価を行います．　環境としてはGoogle が提供する Google Colaboratory上でおこないます． GPU上で処理を行うため，colaboratoryの[ランタイム]->[ランタイムのタイプを変更]からハードウェアアクセラレータをGPUにしてください．

Goolge Colaboratory上にChainerとCupyをインストールします．Data augmentationには，ChainerCVで用意されているクラスを利用するので，ChainerCVのインストールも行います．

In [ ]:
!curl https://colab.chainer.org/install | sh -
!pip install  chainercv 









Chainerでニューラルネットワークを学習するために必要なモジュールや関数をインポートします．また，data augmentationを行うために，画像変換にopencvを利用します．opencvのモジュールであるcv2も合わせてインポートします．

In [ ]:
import numpy as np
import time

import chainer
from chainer import cuda,optimizers, serializers, Chain, Variable
import chainer.functions as F
import chainer.links as L
from chainer.datasets import TransformDataset
from chainercv import transforms
from functools import partial
import cv2


GPUが利用できるか確認します．

In [ ]:
print('GPU availability:', chainer.cuda.available)
print('cuDNN availablility:', chainer.cuda.cudnn_enabled)

次に学習データを読み込みます．CIFAR10データセットは中規模な一般物体認識のデータセットであり，chainerでは CIFAR10
データセットを取得し，学習するためのフォーマットに変換してくれます．データセットには学習用とテスト用のデータに別れており，それぞれtrain_dataset, test_datasetとします．　学習データについて，平均と標準偏差を求めておきます．

In [ ]:
#train_dataset, test_dataset = chainer.datasets.get_cifar10(scale=255.)
train_dataset, test_dataset = chainer.datasets.get_cifar10()
mean = np.mean([x for x, _ in train_dataset], axis=(0, 2, 3))
std = np.std([x for x, _ in train_dataset], axis=(0, 2, 3))
print (len(train_dataset), len(test_dataset))

Data augmentationを行う関数をtransformとして定義します．この関数は学習時のみ利用します．画像を左右反転させるrandom_flip関数，画像を拡大縮小するrandom_expand関数，画像を一定サイズに切り出すrandom_crop関数を利用します．これらの関数はChainerCVのtransformクラスに定義されています．

In [ ]:
def transform(  inputs, mean, std, expand_ratio=1.0, crop_size=(32, 32), train=True):
    img, label = inputs
    img = img.copy()

    # Standardization
#    img -= mean[:, None, None]
#    img /= std[:, None, None]

    if train:
        # Random flip
        img = transforms.random_flip(img, x_random=True)
        # Random expand
        if expand_ratio > 1:
            img = transforms.random_expand(img, max_ratio=expand_ratio)
        # Random crop
        if tuple(crop_size) != (32, 32):
            img = transforms.random_crop(img, tuple(crop_size))

    return img, label


先ほど定義したtransform関数を学習用のtrain_transform，評価用のvalid_transformとします．学習用のtrain_transformには拡大縮小の比率を1.2倍，切り出しサイズを28x28とするように引数で指定します．TransformDatasetクラスは引数に与えたデータセットを同じく引数に与えたdata augmentationの関数に与えて変形させていきます．変形処理はtrain_datasetを呼び出すタイミングで逐次行われます． Partial関数は，第一引数の関数を第二引数以降の引数を与えて随時呼び出す高階関数です．

In [ ]:
train_transform = partial( transform, mean=mean, std=std, expand_ratio=1.2, crop_size=[28, 28], train=True)
valid_transform = partial(transform, mean=mean, std=std, train=False)

train_dataset = TransformDataset(train_dataset, train_transform)
test_dataset = TransformDataset(test_dataset, valid_transform)


畳み込みニューラルネットワークを定義します．ここでは，畳み込み層２層，全結合層３層から構成されるネットワークとします．１層目の畳み込み層は入力チャンネル数が１，出力する特徴マップ数が16，畳み込むフィルタサイズが3x3です．２層目の畳み込み層は入力チャネル数が16．出力する特徴マップ数が32，畳み込むフィルタサイズは同じく3x3です．１つ目の全結合層は入力ユニット数は不定とし，出力は1024としています．次の全結合層入力，出力共に1024，出力層は入力が1024，出力が10です．これらの各層の構成を\__init\__関数で定義します．
次に，\__call\__関数では，定義した層を接続して処理するように記述します．\__call\__関数の引数xは入力データです．それを\__init\__関数で定義したconv1に与え，その出力を活性化関数であるrelu関数に与えます．そして，その出力をmax_pooling_2dに与えて，プーリング処理結果をhとして出力します．hはconv2に与えられて畳み込み処理とプーリング処理を行います．そして，出力hをl1に与えて全結合層の処理を行います．最終的にl3の全結合層の処理を行った出力hを戻り値としています．


In [ ]:
class CNN(Chain):
    def __init__(self):
        super(CNN, self).__init__(
            conv1 = L.Convolution2D(3, 16, 3, pad=1), 
            conv2 = L.Convolution2D(16, 32, 3, pad=1), 
            l1 = L.Linear(None, 1024),
            l2 = L.Linear(1024, 1024),
            l3 = L.Linear(1024, 10)
        )
    def __call__(self, x):
        h = F.max_pooling_2d(F.relu(self.conv1(x)), 2)
        h = F.max_pooling_2d(F.relu(self.conv2(h)), 2)
        h = F.relu(self.l1(h))
        h = F.relu(self.l2(h))
        h = self.l3(h)
        return h

畳み込みネットワークモデルを定義します．学習を行う際の最適化方法としてモーメンタムSGD(モーメンタム付き確率的勾配降下法）を利用します．また，学習率を0.05として引数に与えます．そして，最適化方法のsetup関数にネットワークモデルを与えます．ここでは，GPUで学習を行うために，modelをGPUに送るto_gpu関数を利用しています．また，GPUに対応した行列演算モジュールのcupyを呼び出しています．

In [ ]:
gpu_id = 0 
xp = cuda.cupy
model = CNN()
model.to_gpu(gpu_id)

learnrate = 0.05
optimizer = chainer.optimizers.MomentumSGD(learnrate)
optimizer.setup(model)
optimizer.add_hook(chainer.optimizer.WeightDecay(5e-4))


１回の誤差を算出するデータ数（ミニバッチサイズ）を32，学習エポック数を100とします．CIFAR10の学習データサイズを取得し，１エポック内における更新回数を求めます．まず，各エポックにおいて学習データのaugmentationを行います．学習率は25エポックごとに0.5倍して徐々に小さくしていくようにします．学習データは毎エポックでランダムに利用するため，numpyのpermutationという関数を利用します．各更新において，学習用データと教師データをそれぞれxとtとし，to_gpu関数でGPUに転送します．学習モデルにxを与えて各クラスの確率yを取得します．各クラスの確率yと教師ラベルtとの誤差をsoftmax coross entropy誤差関数で算出します．また，認識精度も算出します．そして，誤差をbackward関数で逆伝播し，ネットワークの更新を行います．

In [ ]:
batch_size = 128
epoch_num = 100

start = time.time()
for epoch in range(epoch_num):
        dataset_x = []
        dataset_y =[]
        for train in train_dataset:
            dataset_x.append(train[0])
            dataset_y.append(train[1])
  
        train_x = xp.asarray(dataset_x, xp.float32)
        train_y = xp.asarray(dataset_y, xp.int32)

        train_data_num = train_x.shape[0]
        iter_one_epoch = int(train_x.shape[0]/batch_size)
  
        num_iter = 0
        sum_loss = 0
        sum_accuracy = 0
        if (epoch+1) % 25 == 0 :
            optimizer.lr *= 0.5
        perm = xp.random.permutation(train_data_num)
        for i in range(0, train_data_num, batch_size):
                x = Variable(cuda.to_gpu(train_x[perm[i:i+batch_size]]))
                t = Variable(cuda.to_gpu(train_y[perm[i:i+batch_size]]))
                y = model(x)        
                model.zerograds()
                loss = F.softmax_cross_entropy(y, t)
                acc = F.accuracy(y, t)
                loss.backward()
                optimizer.update()
                sum_loss += loss.data
                sum_accuracy += acc.data
                num_iter +=1
        elapsed_time = time.time() - start
        print("epoch: {}, mean loss: {}, mean accuracy: {},  elapsed_time :{}".format(epoch+1, sum_loss/num_iter, sum_accuracy/num_iter, elapsed_time))
        
        if (epoch+1) % 10 == 0:
            chainer.serializers.save_hdf5('caifr10_epoch{}.npz'.format(epoch), model, compression=6)
    
    

学習できたネットワークモデルを利用して評価を行います．

In [ ]:
chainer.config.train=False

cnt = 0
dataset_x = []
dataset_y =[]
for test in test_dataset:
    dataset_x.append(transforms.resize(test[0], [28,28]))
    dataset_y.append(test[1])
test_x = xp.asarray(dataset_x, xp.float32)
test_y = xp.asarray(dataset_y, xp.int32)
print (test_x.shape, test_y.shape)
  
test_data_num = test_x.shape[0]
for i in range(0, test_data_num):
       x = Variable(cuda.to_gpu(test_x[i].reshape(1,3,28,28)))
       t = test_y[i]
       y = model(x)        
       y = np.argmax(y.data[0])
       if t == y:
           cnt += 1
print("test accuracy: {}".format(cnt/test_data_num))
    
    

ネットワークを３層の畳み込み層と２層の全結合層から構成されたLeNetベースの構造に変えます．畳み込み層のフィルタサイズは5x5，全結合層のユニット数は4096です．また，畳み込み層と全結合層の間にはSpatial Pyramid Poolingという特殊なプーリング層を用います．

In [ ]:
class LeNet5(chainer.Chain):

    def __init__(self, n_class=10):
        super(LeNet5, self).__init__()
        with self.init_scope():
            self.conv1 = L.Convolution2D(3, 32, 5, stride=1, pad=2)
            self.conv2 = L.Convolution2D(32, 32, 5, stride=1, pad=2)
            self.conv3 = L.Convolution2D(32, 64, 5, stride=1, pad=2)
            self.fc4 = L.Linear(None, 4096)
            self.fc5 = L.Linear(4096, n_class)

    def __call__(self, x):
        h = F.max_pooling_2d(F.relu(self.conv1(x)), 3, stride=2)
        h = F.max_pooling_2d(F.relu(self.conv2(h)), 3, stride=2)
        h = F.relu(self.conv3(h))
        h = F.spatial_pyramid_pooling_2d(h, 3, F.MaxPooling2D)
        h = F.dropout(F.relu(self.fc4(h)), ratio=0.5)
        h = self.fc5(h)
        return h

LeNet5をGPUに送ります．そして，最適化にLeNetを引数として与えます．学習率は0.01とします．

In [ ]:
gpu_id = 0 
xp = cuda.cupy
model = LeNet5()
model.to_gpu(gpu_id)

learnrate = 0.01
optimizer = chainer.optimizers.MomentumSGD(learnrate)
optimizer.setup(model)
optimizer.add_hook(chainer.optimizer.WeightDecay(5e-4))

LeNetによる学習を行います．上記にある学習処理を実行します．（注意：この下のコードではありません．）

次にネットワーク構造をVGGにします．VGGは畳み込み層が13層，全結合層が3層です．畳み込み層のフィルタサイズは3x3です．オリジナルのVGGと異なり，学習が収束しやすくするために各畳み込み層後にBatch Normalizationを適用します．

In [ ]:
class VGG(chainer.Chain):

    def __init__(self, n_class=10):
        super(VGG, self).__init__()
        with self.init_scope():
            self.conv1_1 = L.Convolution2D(None, 64, 3, pad=1)
            self.bn1_1 = L.BatchNormalization(64)
            self.conv1_2 = L.Convolution2D(64, 64, 3, pad=1)
            self.bn1_2 = L.BatchNormalization(64)

            self.conv2_1 = L.Convolution2D(64, 128, 3, pad=1)
            self.bn2_1 = L.BatchNormalization(128)
            self.conv2_2 = L.Convolution2D(128, 128, 3, pad=1)
            self.bn2_2 = L.BatchNormalization(128)

            self.conv3_1 = L.Convolution2D(128, 256, 3, pad=1)
            self.bn3_1 = L.BatchNormalization(256)
            self.conv3_2 = L.Convolution2D(256, 256, 3, pad=1)
            self.bn3_2 = L.BatchNormalization(256)
            self.conv3_3 = L.Convolution2D(256, 256, 3, pad=1)
            self.bn3_3 = L.BatchNormalization(256)
            self.conv3_4 = L.Convolution2D(256, 256, 3, pad=1)
            self.bn3_4 = L.BatchNormalization(256)

            self.fc4 = L.Linear(None, 1024)
            self.fc5 = L.Linear(1024, 1024)
            self.fc6 = L.Linear(1024, n_class)

    def __call__(self, x):
        h = F.relu(self.bn1_1(self.conv1_1(x)))
        h = F.relu(self.bn1_2(self.conv1_2(h)))
        h = F.max_pooling_2d(h, 2, 2)
        h = F.dropout(h, ratio=0.25)

        h = F.relu(self.bn2_1(self.conv2_1(h)))
        h = F.relu(self.bn2_2(self.conv2_2(h)))
        h = F.max_pooling_2d(h, 2, 2)
        h = F.dropout(h, ratio=0.25)

        h = F.relu(self.bn3_1(self.conv3_1(h)))
        h = F.relu(self.bn3_2(self.conv3_2(h)))
        h = F.relu(self.bn3_3(self.conv3_3(h)))
        h = F.relu(self.bn3_4(self.conv3_4(h)))
        h = F.max_pooling_2d(h, 2, 2)
        h = F.dropout(h, ratio=0.25)

        h = F.dropout(F.relu(self.fc4(h)), ratio=0.5)
        h = F.dropout(F.relu(self.fc5(h)), ratio=0.5)
        h = self.fc6(h)
        return h

In [ ]:
gpu_id = 0 
xp = cuda.cupy
model = VGG()
model.to_gpu(gpu_id)

learnrate = 0.01
optimizer = chainer.optimizers.MomentumSGD(learnrate)
optimizer.setup(model)
optimizer.add_hook(chainer.optimizer.WeightDecay(5e-4))

## 課題　
###以下の課題に取り組みましょう

1  Data augmentationのパラメータを変えてみましょう．

　partial関数の引数に与えている拡大縮小の倍率を1.2から1.5に変えましょう

　次に，transform関数でコメントアウトしている平均と標準偏差を利用した処理を行うように修正しましょう
  


2  最適化の方法をAdamに変えて実験しましょう．


3  学習率を変えましょう．

　学習率を0.05から0.01に修正しましょう

　次に，学習率の減衰率を0.5から0.1にしましょう
 